In [ ]:
import boto3

import cv2
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import matplotlib
%matplotlib inline

In [ ]:
# Amazon Textract client
textract = boto3.client(
    service_name='textract',
    region_name='eu-west-2',
    aws_access_key_id=None,
    aws_secret_access_key=None
)

In [ ]:
# Get the image bytes
with open("images/lazy_dog.jpg", 'rb') as image:
    image_bytes = image.read()
    
    
# Call API
result = textract.detect_document_text(
    Document={'Bytes': image_bytes}
)

In [ ]:
result

In [ ]:
# Print detected text
for item in result["Blocks"]:
    if item["BlockType"] == "LINE":
        print(item["Geometry"]["Polygon"])
        print(item["Text"])

In [ ]:
# Display the image and overlay it with the extracted text.
image = cv2.imread("images/lazy_dog.jpg")
ax = plt.imshow(image)

for item in result["Blocks"]:
    if item["BlockType"] == "LINE":
    
        # Extract the (x,y) coordinates from [{x0,y0},...,{x3,y3}]
        bbox = item["Geometry"]["Polygon"]
        vertices = [
            (bbox[i]["X"] * image.shape[1], bbox[i]["Y"] * image.shape[0]) 
            for i in range(0, 4)
        ] # multiply by width / height as given as proportion
        
        # Draw on the polygon
        patch = Polygon(vertices, closed=True, fill=False, linewidth=2, color='y')
        ax.axes.add_patch(patch)

        # Add the text
        text = item["Text"]
        plt.text(vertices[0][0], vertices[0][1], text, fontsize=20, va="top")

plt.show()